In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, os

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow import keras

import warnings
warnings.filterwarnings("ignore")

In [4]:
df = pd.read_csv("train_date_pid.csv")
df.head()

,review_rating,number_of_photos,helpful_vote,reviewer_ID,fake_asin,fake_review,product_ID,review_ID,review,month,is_weekday,polarity,subjectivity,labeled_product_id
0,5,0,0,229286,0,0,211,405773,works well perfect fit slim low profile frame ...,August,0,0.433333,0.533333,0
1,5,0,0,481853,0,0,140,281967,good item fast delivery puts light right need,November,1,0.396429,0.608929,0
2,1,0,1,359300,0,0,304,103693,stay connected stay connected keeps dropping s...,July,1,-0.083333,0.400000,0
3,5,0,0,79568,0,0,93,65229,nice item buy bubbler need cups nice youcan mo...,June,1,0.633333,0.880556,0
4,5,0,0,1066577,0,0,309,75165,5 stars 5 stars,January,0,0.000000,0.000000,0


In [5]:
X = df.drop(['reviewer_ID', 'fake_asin', 'product_ID', 'review_ID', 'review', 'fake_review'], axis = 1)

# get dummies
X = pd.get_dummies(X, columns=['month'])
X = pd.get_dummies(X, columns=['labeled_product_id'])
X = X.drop(['labeled_product_id_0'], axis = 1)

# scaling
stander = StandardScaler()

X = stander.fit_transform(X)

In [6]:
y = df['fake_review'] # 标签数据集

In [7]:
model = keras.models.Sequential()


model.add(keras.layers.Dense(2048, activation='relu', input_dim=X.shape[1]))

model.add(keras.layers.Dense(1024, activation='relu'))

model.add(keras.layers.Dense(512, activation='relu'))

model.add(keras.layers.Dense(256, activation='relu'))

model.add(keras.layers.Dense(128, activation='relu'))

model.add(keras.layers.Dense(1, activation='sigmoid')) 

In [8]:
model.compile(loss=keras.losses.BinaryFocalCrossentropy(), optimizer='adam', metrics=['accuracy'])

In [9]:
# compute the class weights
n_samples = len(y)
n_classes = len(np.unique(y))
class_counts = np.bincount(y)
class_weights = {i: n_samples / (n_classes * class_counts[i]) for i in range(n_classes)}

# fit the model with sample weights
sample_weights = np.array([class_weights[label] for label in y])

model.fit(X, y, sample_weight=sample_weights, epochs = 20, validation_split=0.2, batch_size=64)

Epoch 1/20
1796/4899 [=========>....................] - ETA: 1:40 - loss: 0.1534 - accuracy: 0.8595

KeyboardInterrupt: 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 12343)

In [ ]:
from sklearn.metrics import confusion_matrix
# Get the predicted labels and true labels
y_pred = model.predict(X_test)

def my_fuc(x):
    if x < 0.65:
        return 0
    else:
        return 1
    
y_pred = np.array([my_fuc(x) for x in y_pred])

# Compute the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Print the confusion matrix
print(cm)

from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

In [ ]:
df_test = pd.read_csv("test_df_pid.csv")
df_test_sample = df_test.drop(['reviewer_ID', 'product_ID', 'review_ID', 'review', 'fake_asin'], axis = 1)
df_test_sample = pd.get_dummies(df_test_sample, columns=['month'])

df_test_sample = pd.get_dummies(df_test_sample, columns=['labeled_product_id'])
df_test_sample = df_test_sample.drop(['labeled_product_id_0'], axis = 1)

stander = StandardScaler()
df_test_sample = stander.fit_transform(df_test_sample)
df_test_sample

In [ ]:
predictions = model.predict(df_test_sample)
predictions

In [ ]:
df_test["fake_review"] = predictions

In [ ]:
output = df_test[["review_ID", "fake_review"]]

In [ ]:
output['fake_review'] = output['fake_review'].apply(lambda x: 0 if x < 0.65 else 1)

In [ ]:
output.to_csv('NN.csv', index=False)